## BERT 파인튜닝

# 📌 자연어 처리 텍스트 분류 파이프라인 전체 흐름 이해하기

이 코드는 영화 리뷰 텍스트가 **긍정인지 부정인지 분류하는 자연어 처리(NLP)** 예제입니다.  

---

## ✅ 단계 1: 문제 정의 - 텍스트를 분류하는 모델 만들기
- 목적: 영화 리뷰를 보고 **긍정(좋다)**인지 **부정(싫다)**인지 자동으로 판단하는 모델을 만든다.
- 입력: 영화 리뷰 텍스트 (예: `"This movie was awesome!"`)
- 출력: 0 (부정) 또는 1 (긍정)

> 🎯 예를 들면, 모델에게 `"It was amazing!"`이라는 문장을 주면 → `1(긍정)`이라고 판단하게 만드는 것이 목표입니다.

---

## ✅ 단계 2: 데이터 준비 - 영화 리뷰 데이터를 가져오기
- 사용하는 데이터셋: **IMDB 영화 리뷰 데이터셋**
- 예제에서는 **20개 리뷰만 샘플로 사용**하고, 이 중 80%는 학습용, 20%는 테스트용으로 나눕니다.

> 🧩 왜 데이터를 나눌까?
> - **학습 데이터**로 모델을 훈련시키고  
> - **테스트 데이터**로 모델이 잘 학습되었는지 검증합니다.

---

## ✅ 단계 3: 텍스트 전처리 - 단어를 숫자로 바꾸기
- 컴퓨터는 글자가 아니라 숫자를 이해합니다.
- 그래서 `"I loved the movie"` 같은 문장을 숫자 토큰으로 바꿔야 합니다.
- 이 작업을 도와주는 것이 바로 **토크나이저(Tokenzier)** 입니다.

> ✨ 비유하자면, 문장을 ‘레고 블록’처럼 잘게 쪼개고, 각각에 번호를 붙이는 과정입니다.

---

## ✅ 단계 4: 모델 준비 - 사전학습된 똑똑한 모델 불러오기
- 우리는 처음부터 모델을 만드는 게 아니라, 이미 영어를 어느 정도 학습해둔 모델(**DistilBERT**)을 가져옵니다.
- 이 모델에 "긍정/부정 분류"라는 **새로운 임무를 학습시키는 것**입니다. 이를 **파인튜닝(Fine-tuning)**이라고 합니다.

> 🧠 쉽게 말해, "기초 영어는 이미 배운 친구에게 감정 분석이라는 새 과목을 가르치는 것"입니다.

---

## ✅ 단계 5: 모델 훈련 - 데이터로 학습시키기
- 앞에서 준비한 텍스트(숫자 형태)와 라벨(긍정/부정)을 가지고 모델을 학습합니다.
- 학습은 30번 반복해서 진행됩니다 (30 에폭).

> 🚀 학습이란? 모델이 "이 문장이 긍정인지 부정인지" 스스로 맞추도록 훈련시키는 과정입니다.

---

## ✅ 단계 6: 예측하기 - 새로운 문장에 대해 모델이 판단하도록 하기
- 학습이 끝난 모델에 새로운 문장을 넣어보면, 긍정인지 부정인지 판단해줍니다.
- 예: `"It was great to see John Ritter"` → 모델이 **긍정**이라고 판단하면 `1`을 반환합니다.

---

## 🔚 마무리 요약
이 코드는 다음과 같은 전체 구조를 따릅니다:



In [1]:
##세션 다시 시작 필요
!pip install -q transformers datasets
# 🤖 Hugging Face 라이브러리 설치
# transformers: 사전학습된 NLP 모델을 쉽게 사용할 수 있게 해주는 라이브러리
# datasets: 다양한 자연어처리용 데이터셋을 손쉽게 불러올 수 있는 라이브러리
!pip install -U "datasets<=2.18.0" "fsspec<=2023.6.0"


In [22]:
# ✅ 1. 데이터 로드
from datasets import load_dataset
dataset = load_dataset("imdb", split="train[:20]").train_test_split(test_size=0.2)
# 🔍 포인트: IMDB 영화 리뷰 데이터셋 중 20개 샘플만 가져와서 학습용/평가용으로 8:2 비율로 나눕니다
# - 'train[:20]': 전체 학습 데이터 중 앞에서 20개만 사용 (예제 실행용으로 작게 설정한 것)
# - train_test_split: 데이터를 학습용(train)과 테스트용(test)으로 자동 분리

print(dataset["train"][1])
# 🔍 실제로 어떤 데이터가 들어있는지 확인해보는 코드입니다 (리뷰 텍스트와 라벨 포함)
# - label: 0 = 부정적인 리뷰, 1 = 긍정적인 리뷰


{'text': 'It was great to see some of my favorite stars of 30 years ago including John Ritter, Ben Gazarra and Audrey Hepburn. They looked quite wonderful. But that was it. They were not given any characters or good lines to work with. I neither understood or cared what the characters were doing.<br /><br />Some of the smaller female roles were fine, Patty Henson and Colleen Camp were quite competent and confident in their small sidekick parts. They showed some talent and it is sad they didn\'t go on to star in more and better films. Sadly, I didn\'t think Dorothy Stratten got a chance to act in this her only important film role.<br /><br />The film appears to have some fans, and I was very open-minded when I started watching it. I am a big Peter Bogdanovich fan and I enjoyed his last movie, "Cat\'s Meow" and all his early ones from "Targets" to "Nickleodeon". So, it really surprised me that I was barely able to keep awake watching this one.<br /><br />It is ironic that this movie is a

In [23]:
 # ✅ 2. 모델 및 토크나이저 준비
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
# 🔍 포인트: 토크나이저란? → 텍스트를 숫자 토큰으로 바꾸는 역할
# "나는 배가 고프다" → [101, 1432, 2234, 1678, 102] ← 이런 식으로 숫자로 바꿔줘야 모델이 이해 가능

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")
# 🔍 distilbert-base-uncased: BERT를 경량화한 모델 (빠르고 정확도도 준수)
# 포인트: 사전학습(pretrained)된 모델이므로, 적은 데이터로도 빠르게 학습 가능

# ✅ 3. 데이터 전처리 (텍스트를 숫자 토큰으로 변환)
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding=True)
# 🔍 포인트: tokenizer 함수로 모든 리뷰 텍스트를 숫자 토큰으로 바꿉니다
# - truncation=True: 문장이 너무 길면 자르고
# - padding=True: 짧은 문장은 0으로 채워 길이를 맞춰줍니다

dataset = dataset.map(tokenize, batched=True)
# 🔍 map 함수: 전체 데이터셋에 대해 위 tokenize 함수를 한꺼번에 적용
# - batched=True: 여러 데이터를 동시에 처리해서 속도 향상


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [24]:
# ✅ 4. 훈련 설정 및 Trainer 구성
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir="test",  # 모델이 저장될 경로
    per_device_train_batch_size=8,  # 학습 시 배치 크기 (GPU 하나당 8개씩 처리)
    num_train_epochs=30  # 에폭 수 (데이터셋을 30번 반복해서 학습)
)

trainer = Trainer(
    model=model,  # 사용할 모델
    args=args,  # 훈련 설정값
    train_dataset=dataset["train"],  # 학습용 데이터셋
    eval_dataset=dataset["test"],  # 검증용 데이터셋
)
# 🔍 포인트: Trainer는 Hugging Face에서 제공하는 편리한 훈련 도구
# 복잡한 학습 코드 없이도 손쉽게 훈련 가능

In [25]:
# ✅ 5. 파인튜닝 실행 (모델 학습)


trainer.train()
# 🔍 포인트: 위 한 줄만으로 모델이 훈련됨! (사전학습된 모델을 우리가 준비한 데이터에 맞춰 미세조정)

Step,Training Loss


TrainOutput(global_step=60, training_loss=0.03878935178120931, metrics={'train_runtime': 31.9693, 'train_samples_per_second': 15.014, 'train_steps_per_second': 1.877, 'total_flos': 63584351354880.0, 'train_loss': 0.03878935178120931, 'epoch': 30.0})

In [26]:
# ✅ 학습된 모델로 실제 예측 수행
text = "It was great to see some of my favorite stars of 30 years ago including John Ritter"
inputs = tokenizer(text, return_tensors="pt").to("cuda")
# 🔍 포인트: 새 문장을 토큰화한 후, GPU로 보내줌 (".to('cuda')" 필수)
# "pt": PyTorch 텐서 형식으로 변환

output = model(**inputs)
label = output.logits.argmax(-1).item()
# 🔍 포인트: 모델 출력(logits)은 [부정 점수, 긍정 점수] 형태의 벡터
# argmax(-1): 가장 높은 점수의 인덱스 (0 또는 1)를 예측값으로 사용

print("긍정" if label == 1 else "부정")
# 🔍 최종 결과 출력: 긍정인지 부정인지 해석해서 보여줌

부정


# GPT-2 파인튜닝

| 항목                     | `distilbert-base-uncased` | `gpt2`                    |
| ---------------------- | ------------------------- | ------------------------- |
| 🔢 모델 타입               | Encoder (BERT 계열)         | Decoder (GPT 계열)          |
| 📦 사전학습 목적             | 마스킹된 언어 모델(Masked LM)     | 오토리그레시브 언어 생성(Next token) |
| 🧠 파라미터 수              | 약 66M                     | 약 124M                    |
| 🏗️ 레이어 수              | 6                         | 12                        |
| 🧩 히든 크기 (Hidden Size) | 768                       | 768                       |
| 🔁 어텐션 헤드 수            | 12                        | 12                        |
| 📏 최대 시퀀스 길이           | 512 tokens                | 1024 tokens               |
| 📄 토크나이저 타입            | WordPiece (BERT형)         | Byte-Pair Encoding (BPE)  |
| 💬 생성 능력               | ❌ (비생성형, 분류에 적합)          | ✅ (텍스트 생성에 최적)            |
| ⚙️ 사용 예시               | 텍스트 분류, 감정 분석 등           | 텍스트 생성, 요약, 질문응답 등        |
| ⚡ 연산 효율성               | 빠름 (BERT의 경량화 버전)         | 느림 (텍스트 생성 연속 수행 필요)      |



# 🧠 GPT-2 파인튜닝, 전체 흐름 한눈에 보기

## 🎯 목표
GPT-2에게 **우리만의 질문-답변 방식**을 가르쳐서, 원하는 스타일로 대답하게 만드는 것!

## 🛠️ 사용하는 기술
- **GPT-2**: 텍스트 생성에 특화된 언어 모델
- **LoRA**: 모델 전체를 바꾸지 않고 일부만 효율적으로 조정하는 기술
- **Trainer**: 학습을 쉽게 해주는 자동화 도구

## 📋 전체 단계 요약

1. **도구 설치**: 필요한 라이브러리 불러오기
2. **모델 로드**: GPT-2와 토크나이저 불러오기
3. **LoRA 설정**: 빠르고 가볍게 학습할 수 있도록 구성
4. **데이터 준비**: 질문-답변 예시 입력
5. **토큰화**: 텍스트 → 숫자 변환
6. **학습 설정**: 반복 횟수, 배치 크기 등 지정
7. **학습 실행**: 모델에게 예시 보여주며 훈련
8. **결과 확인**: 새 질문 넣어보기

## 💡 핵심 요약
- GPT-2는 원래 똑똑함 → 우리는 "내 스타일(내데이터)"로 조금만 바꿈
- LoRA 덕분에 가볍고 빠르게 가능



In [7]:
# STEP 1: 환경 설정
!pip install -q transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.3 MB/s eta 0:00:00


In [8]:
# ✅ STEP 2: 기본 라이브러리 임포트
# 모델 학습에 필요한 필수 라이브러리들을 불러옵니다.

import torch  # PyTorch: 딥러닝 프레임워크
from datasets import Dataset  # 텍스트 데이터를 Dataset 객체로 변환하는 데 사용
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType  # LoRA 기반 파인튜닝 지원

In [9]:

# ✅ STEP 3: 모델 및 토크나이저 로드
# Hugging Face에서 제공하는 GPT-2 모델과 토크나이저를 불러옵니다.

model_id = "gpt2"  # 사용할 모델 지정 (GPT-2)

# 토크나이저는 텍스트를 숫자 시퀀스로 바꿔주는 도구입니다.
tokenizer = AutoTokenizer.from_pretrained(model_id)

# GPT-2는 pad_token이 기본적으로 없으므로, eos_token(문장 종료 토큰)을 pad_token으로 설정합니다.
tokenizer.pad_token = tokenizer.eos_token

# 사전학습된 GPT-2 모델을 불러옵니다. 텍스트 생성 작업에 최적화된 모델입니다.
model = AutoModelForCausalLM.from_pretrained(model_id)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [12]:
# ✅ STEP 4: LoRA 설정
# [포인트] LoRA(Low-Rank Adaptation)는 대규모 모델을 가볍게 미세조정할 수 있게 해주는 기술입니다.
# 전체 모델을 학습시키지 않고, 일부 작은 파라미터만 추가로 학습하기 때문에 훨씬 효율적입니다.

lora_config = LoraConfig(
    r=4,  # 랭크: LoRA 내부 차원. 작을수록 가볍고 빠름
    lora_alpha=16,  # 학습 안정성을 위한 스케일링 계수
    target_modules=["c_attn"],  # GPT-2에서 LoRA를 적용할 레이어. 'c_attn'은 Attention 모듈의 핵심 부분입니다.
    # 🔍 GPT-2의 레이어들
    # - "c_attn": 쿼리, 키, 밸류를 생성하는 핵심 어텐션 입력 레이어
    # - "c_proj": 어텐션 출력 벡터를 변환하는 투영 레이어
    # - "q_attn": self-attention 중 쿼리 연산에 관여하는 부분 (GPT-J 등 일부 구조에 존재)
    # - "mlp.c_fc": 피드포워드 네트워크의 첫 번째 선형 계층 (MLP의 입력 부분)
    # - "mlp.c_proj": MLP의 출력 부분
    lora_dropout=0.05,  # 드롭아웃 적용 (과적합 방지)
    bias="none",  # bias 파라미터는 학습하지 않음
    task_type=TaskType.CAUSAL_LM,  # 작업 유형: 언어 생성 (Causal Language Modeling)
)

# ✅ 모델에 LoRA 설정을 적용합니다.
# 기존 GPT-2 위에 LoRA 구조를 얹어, 일부 파라미터만 학습 가능하게 만듭니다.
# 추론 시에는 원래 모델과 LoRA가 함께 사용되므로 merge 없이도 작동합니다.
model = get_peft_model(model, lora_config)

# 실제로 학습 가능한 파라미터 수를 출력해봅니다 (LoRA 파라미터만 학습되므로 매우 적습니다).
model.print_trainable_parameters()

trainable params: 147,456 || all params: 124,587,264 || trainable%: 0.1184


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [13]:

# ✅ STEP 5: 간단한 학습 데이터셋 정의
# 질문-답변 형태의 짧은 데이터셋을 정의합니다.
# 실습 목적이므로 간단한 예시로 구성되어 있습니다.

data = {
    "text": [
        "### 질문: 우리집 강아지 이름은?\n### 답변: 순둥이",
        "### 질문: 바다는 왜 파란가요?\n### 답변: 햇빛의 산란",
    ]
}

# Hugging Face Dataset 객체로 변환
dataset = Dataset.from_dict(data)


In [14]:

# ✅ STEP 6: 토크나이즈 함수
# [포인트] 텍스트 데이터를 모델에 넣기 위해 숫자 형태(토큰 ID)로 변환합니다.
# padding과 truncation을 설정해서 입력 길이를 일정하게 맞춰줍니다.

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

# 데이터셋에 토크나이즈 함수를 적용하여 숫자형 시퀀스로 변환
tokenized_dataset = dataset.map(tokenize_function)


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [15]:
# ✅ STEP 7: 데이터 콜레이터
# [개념] 데이터 콜레이터는 배치로 묶을 때 패딩, 마스킹 등을 자동으로 처리해주는 도구입니다.
# GPT처럼 다음 단어를 예측하는 방식에서는 MLM(False)로 설정합니다.

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


데이터 콜레이터란?
DataCollatorForLanguageModeling은 모델에 데이터를 넣기 직전에 문장들을 자동으로 정리해주는 도구입니다.
모델이 문장을 한 번에 여러 개 처리하려면 길이를 맞춰야 하는데, 그 과정을 **패딩(padding)**이라고 해요.
또, 어떤 위치를 학습해야 할지 표시해주는 **마스킹(masking)**도 필요하죠.

이 과정을 직접 처리하지 않아도 되게끔, 데이터 콜레이터가 알아서 처리해 줍니다.

In [16]:
# ✅ STEP 8: 트레이닝 설정
# [포인트] 모델 학습에 필요한 하이퍼파라미터들을 설정합니다.
# output_dir: 결과 저장 경로 / 배치 크기 / 에폭 수 등 설정

training_args = TrainingArguments(
    output_dir="./results",  # 결과 디렉토리
    per_device_train_batch_size=10,  # GPU나 CPU 1개당 배치 사이즈
    num_train_epochs=50,  # 데이터 전체를 몇 번 반복해서 학습할지 (여기선 10번)
    logging_steps=1,  # 1스텝마다 로그 출력 (학습 진행 상황 확인)
    save_strategy="no",  # 중간 저장 생략 (데모 목적이므로)
    fp16=False,  # GPU 없이 CPU로 학습 시 False 설정
)


✅ step과 epoch의 차이 정리

- **step은 iteration과 동일한 의미**입니다. 즉, 한 번의 파라미터 업데이트를 뜻합니다.
- 모델이 **한 배치(batch)**를 보고 학습하는 것이 1 step (1 iteration)입니다.
- 반면 **epoch**은 전체 데이터를 한 바퀴 학습한 것을 의미합니다.
- 예: 데이터 100개, 배치 크기 10이면 → 10 step = 1 epoch입니다.
- 정리하면, **여러 step이 모여 하나의 epoch을 구성**합니다.


In [17]:
# ✅ STEP 9: 트레이너 설정 및 학습 시작
# Hugging Face의 Trainer 클래스를 사용해 학습을 진행합니다.
# 위에서 정의한 모델, 데이터, 설정값을 모두 전달하여 학습을 시작합니다.

trainer = Trainer(
    model=model,  # 학습할 모델
    args=training_args,  # 학습 설정
    train_dataset=tokenized_dataset,  # 학습 데이터
    tokenizer=tokenizer,  # 텍스트 디코딩용
    data_collator=data_collator,  # 배치 구성 도우미
)

# 학습 시작!
trainer.train()

<ipython-input-17-f7cce3f05d40>:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sungkun118 (sungkun118-d) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
1,3.337900
2,3.594800
3,3.941200
4,3.203500
5,3.161000
6,3.295300
7,3.848200
8,3.160000
9,3.317400
10,3.240500


TrainOutput(global_step=50, training_loss=3.340850381851196, metrics={'train_runtime': 105.1447, 'train_samples_per_second': 0.951, 'train_steps_per_second': 0.476, 'total_flos': 6543625420800.0, 'train_loss': 3.340850381851196, 'epoch': 50.0})

In [18]:
# "우리집 강아지 이름은?"이라는 질문에 답을 하도록 구성합니다.
input_text = "### 질문:우리집 강아지 이름은?\n### 답변:"

# 입력 문장을 숫자로 바꿔주는 tokenizer를 사용해 모델이 이해할 수 있는 형식으로 변환합니다.
# return_tensors="pt"는 PyTorch 텐서 형태로 반환하겠다는 뜻입니다.
inputs = tokenizer(input_text, return_tensors="pt")

# 모델이 올라가 있는 디바이스(GPU 또는 CPU)를 가져옵니다.
device = model.device

# 입력 데이터도 모델이 있는 디바이스로 옮겨줍니다.
# 그래야 모델과 데이터가 같은 장치에 있어 연산이 가능합니다.
inputs = {k: v.to(device) for k, v in inputs.items()}

# 모델에게 답변을 생성하도록 지시합니다.
# max_new_tokens=50은 최대 50개의 새로운 단어(토큰)를 생성하겠다는 의미입니다.
outputs = model.generate(**inputs, max_new_tokens=50)

# 생성된 답변을 사람이 읽을 수 있는 문자열로 바꿔서 출력합니다.
# skip_special_tokens=True는 시작/종료 같은 특수 기호는 출력하지 않겠다는 의미입니다.
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


### 질문:우리집 강아지 이름은?
### 답변:우�집 강지 �는도 가지 이름 가지 이름


GPT-2 는 LoRA를 적용할만큼 큰 게 아니다.

# GPT2파인튜닝 -lora 제거

In [19]:
# STEP 1: 환경 설정
!pip install -q transformers datasets

# STEP 2: 기본 라이브러리 임포트
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)

# STEP 3: 모델 및 토크나이저 로드
model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_id)
model.resize_token_embeddings(len(tokenizer))  # pad_token 추가 고려

# STEP 4: 학습 데이터 정의
data = {
    "text": [
        "### 질문: 우리집 강아지 이름은?\n### 답변: 순둥이",
        "### 질문: 바다는 왜 파란가요?\n### 답변: 햇빛의 산란",
    ]
}
dataset = Dataset.from_dict(data)

# STEP 5: 토큰화
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function)

# STEP 6: 데이터 콜레이터
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# STEP 7: 트레이닝 설정
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=10,
    num_train_epochs=50,
    logging_steps=1,
    save_strategy="no",
    fp16=False,
)

# STEP 8: 트레이너 설정 및 학습
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

<ipython-input-19-56aa572bc554>:52: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
1,3.353300
2,3.258700
3,2.557400
4,2.369300
5,2.136900
6,1.868400
7,1.589700
8,1.671300
9,1.674400
10,1.193500


TrainOutput(global_step=50, training_loss=0.696427931189537, metrics={'train_runtime': 6.389, 'train_samples_per_second': 15.652, 'train_steps_per_second': 7.826, 'total_flos': 6532300800000.0, 'train_loss': 0.696427931189537, 'epoch': 50.0})

In [20]:

# STEP 9: 추론 예시
input_text = "### 질문: 우리집 강아지 이름은?\n### 답변:"
inputs = tokenizer(input_text, return_tensors="pt")

# 모델과 같은 디바이스로 이동
device = model.device
inputs = {k: v.to(device) for k, v in inputs.items()}

outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


### 질문: 우리집 강아지 이름은?
### 답변: 순둥이 산란 이빛�?
### 답변: 햇빛의 산란 순�


### GPT-2에 LoRA를 사용한게 더 성능이 떨어짐
### 어중간하게 작은 모델은 풀 파인튜닝 하는게 효율적이다

## OPT-350M 파인튜닝- Lora미적용

### 🔍 GPT-2 vs OPT-350M 상세 비교표

| 항목             | GPT-2                                     | OPT-350M (facebook/opt-350m)                      |
|------------------|--------------------------------------------|---------------------------------------------------|
| **개발자**        | OpenAI                                     | Meta (Facebook AI Research)                       |
| **공개 연도**     | 2019                                       | 2022                                              |
| **파라미터 수**    | 약 **1.2억 (small)**~15억 (large)             | 약 **3.5억**                                       |
| **학습 데이터**   | 웹페이지, 위키 등 일반 텍스트 (WebText)       | Open Pretraining Dataset (RoBERTa 등 기반)        |
| **학습 목적**     | 일반 언어 생성                              | 고효율, GPT 스타일 대안 (특히 학계/산업용 목적)     |
| **아키텍처**      | Decoder-only Transformer                  | GPT-2 기반 Decoder-only 구조 (단순하고 효율적)     |
| **라이선스**      | 오픈 (MIT 라이선스 아님)                     | 오픈 (Apache 2.0 라이선스)                         |
| **파인튜닝 지원** | 매우 잘 지원 (가볍고 빠름)                   | Hugging Face와 매우 잘 통합됨                      |
| **사용 용도**     | 데모, 실습용, 기본 생성 테스트에 적합          | 실무 서비스, 연구 실험, 챗봇, Q&A에 적합             |
| **장점**          | 가볍고 빠름, 구조가 단순함                    | 성능 대비 크기 효율 좋고, Colab에서도 잘 동작함       |
| **단점**          | 표현력, 논리성, 길이 제어에서 한계              | 너무 긴 문장에서는 GPT-J 등보다 표현력이 부족할 수 있음 |


In [27]:
# 설치
!pip install -q transformers datasets  # Hugging Face의 트랜스포머 라이브러리와 데이터셋 도구 설치

# ===============================
# 필요한 라이브러리 불러오기
# ===============================
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling
from datasets import Dataset
import torch

# ===============================
# 🔹 사전 학습된 모델 및 토크나이저 불러오기
# ===============================
model_id = "facebook/opt-350m"  # 사용할 사전 학습 언어 모델 ID
tokenizer = AutoTokenizer.from_pretrained(model_id)  # 텍스트를 숫자로 바꾸는 토크나이저 로드
tokenizer.pad_token = tokenizer.eos_token
# OPT 모델은 'pad_token' (빈칸을 채우는 용도)이 기본적으로 정의되어 있지 않습니다.
# 모델이 문장의 길이를 맞춰서 처리할 수 있도록 'pad_token'이 필요합니다.
# 그래서 여기서는 문장의 끝을 나타내는 'eos_token' (end-of-sequence token)을 대신 사용합니다.
# 즉, 빈칸 자리를 eos_token으로 채우도록 설정하는 것입니다.
model = AutoModelForCausalLM.from_pretrained(model_id)  # Causal Language Modeling용 사전 학습 모델 로드
model.config.pad_token_id = tokenizer.pad_token_id  # 모델 설정에도 pad_token ID를 지정해줘야 에러 방지 가능

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/662M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [28]:
# ===============================
# 🔹 학습할 데이터 구성
# ===============================
data = {
    "text": [
        "### 질문: 우리집 강아지 이름은?\n### 답변: 순둥이",  # [예시] 단순 QA 형태의 문장. 이후 이 형식을 기반으로 추가 학습됨
        "### 질문: 바다는 왜 파란가요?\n### 답변: 햇빛의 산란",
        "### 질문 : 왜 비트코인은 오르나요?\n### 답변 : 디지털 금",
    ]
}
dataset = Dataset.from_dict(data)  # Hugging Face Dataset 객체로 변환


In [29]:
# ===============================
# 🔹 토큰화 함수 정의 및 적용
# ===============================
def tokenize(example):
    # [포인트] max_length: 64로 고정 → 긴 문장은 자르고, 짧은 문장은 패딩
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=64)

tokenized_dataset = dataset.map(tokenize)  # 데이터셋 전체에 토큰화 함수 적용

# ===============================
# 🔹 데이터 콜레이터 설정
# ===============================
# 모델에 배치로 넣기 전에 텐서 형태로 묶어주는 역할
# mlm=False → [포인트] 'Causal LM' 방식이므로 MLM(Masked LM)은 사용하지 않음
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)



Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [30]:
# ===============================
# 🔹 학습 하이퍼파라미터 설정
# ===============================
training_args = TrainingArguments(
    output_dir="./results",  # 학습 결과 저장 폴더
    per_device_train_batch_size=1,  # [포인트] 한 번에 하나씩 학습 → 소규모 실습용 설정
    num_train_epochs=50,  # 전체 데이터셋을 50번 반복 학습
    logging_steps=1,  # 매 스텝마다 로그 출력
    save_strategy="no",  # 학습 중 체크포인트 저장하지 않음
    fp16=True,  # GPU에서 float16 사용 여부 (True로 설정하면 메모리 효율 ↑, CPU에서는 False 유지)
    report_to="none",  # 로그 저장 위치 (None으로 설정 시 WandB 등 외부로 전송 안 함)
)

# ===============================
# 🔹 Trainer 객체 구성 및 학습 시작
# ===============================
trainer = Trainer(
    model=model,  # 학습할 모델
    args=training_args,  # 학습 설정
    train_dataset=tokenized_dataset,  # 학습 데이터셋
    tokenizer=tokenizer,  # 토크나이저 (로그 기록이나 디코딩 시 사용)
    data_collator=data_collator,  # 배치 전처리 콜레이터
)

trainer.train()  # 실제 학습 시작


<ipython-input-30-593773abb4ae>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
1,2.910400
2,2.774100
3,2.465000
4,1.444600
5,2.539300
6,1.244600
7,0.479800
8,0.765300
9,2.964000
10,0.358300


TrainOutput(global_step=150, training_loss=0.21509590805818637, metrics={'train_runtime': 37.4287, 'train_samples_per_second': 4.008, 'train_steps_per_second': 4.008, 'total_flos': 17473418035200.0, 'train_loss': 0.21509590805818637, 'epoch': 50.0})

In [31]:
# ===============================
# 🔹 학습된 모델로 텍스트 생성 (추론)
# ===============================
input_text = "### 질문: 우리집 강아지 이름은?\n### 답변:"  # [예시] 질문에 대한 답변을 생성해보는 입력
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)  # 토큰화 및 모델에 넣을 수 있도록 텐서로 변환
outputs = model.generate(**inputs, max_new_tokens=50)  # 최대 50 토큰 길이의 응답 생성
print(tokenizer.decode(outputs[0], skip_special_tokens=True))  # 토큰을 사람이 읽을 수 있는 텍스트로 디코딩하여 출력


### 질문: 우리집 강아지 이름은?
### 답변: 순둥이나요?
### 답변: 순둥이나요?
### 답변: �


In [35]:
input_text = "질문에 대답해줘 비트코인"  # [예시] 질문에 대한 답변을 생성해보는 입력
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)  # 토큰화 및 모델에 넣을 수 있도록 텐서로 변환
outputs = model.generate(**inputs, max_new_tokens=50)  # 최대 50 토큰 길이의 응답 생성
print(tokenizer.decode(outputs[0], skip_special_tokens=True))  # 토큰을 사람이 읽을 수 있는 텍스트로 디코딩하여 출력

질문에 대답해줘 비트코인은 오르나요?
### 답변: 순둥이나요?
### 답변: �


## OPT-350M 파인튜닝- Lora적용

In [36]:
!pip install -q transformers datasets peft accelerate


In [37]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType
import torch

# 사전 학습 모델 로드
model_id = "facebook/opt-350m"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(model_id)
base_model.config.pad_token_id = tokenizer.pad_token_id

# LoRA 설정 적용
lora_config = LoraConfig(
    r=8,  # 🔹 LoRA의 "랭크(rank)" 값입니다.
          # 학습할 파라미터 수를 줄이는 정도를 설정합니다.
          # r이 작을수록 계산이 가벼워지지만, 너무 작으면 성능이 떨어질 수 있습니다.

    lora_alpha=16,  # 🔹 LoRA가 학습한 정보를 얼마나 강하게 모델에 반영할지 정하는 값입니다.
                    # 일종의 "확대 비율"처럼 작용하며, 일반적으로 r과 함께 조정합니다.

    lora_dropout=0.05,  # 🔹 학습 중 일부 정보를 무작위로 버려 과적합을 막는 기술입니다.
                        # 0.05는 5% 확률로 드롭아웃이 일어나도록 설정한 것입니다.

    bias="none",  # 🔹 기존 모델의 편향(bias) 파라미터는 건드리지 않겠다는 뜻입니다.
                  # 즉, 오직 LoRA 레이어만 학습합니다.

    task_type="CAUSAL_LM"  # 🔹 이 설정이 적용될 작업의 유형입니다.
                           # "CAUSAL_LM"은 일반적인 언어 생성 모델(예: GPT)에서 사용됩니다.
)


model = get_peft_model(base_model, lora_config)

# 학습할 데이터 구성
data = {
    "text": [
        "### 질문: 우리집 강아지 이름은?\n### 답변: 순둥이",
    ]
}
dataset = Dataset.from_dict(data)

# 토큰화
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=64)

tokenized_dataset = dataset.map(tokenize)

# 데이터 콜레이터
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 학습 하이퍼파라미터
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    num_train_epochs=50,
    logging_steps=1,
    save_strategy="no",
    fp16=False,
    report_to="none"
)

# Trainer 구성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

<ipython-input-37-ec5ceb5e10c5>:66: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
1,2.945800
2,2.874700
3,2.866000
4,2.721500
5,2.854500
6,2.781300
7,2.754500
8,2.674300
9,2.630100
10,2.632100


TrainOutput(global_step=50, training_loss=2.3774278950691223, metrics={'train_runtime': 11.2579, 'train_samples_per_second': 4.441, 'train_steps_per_second': 4.441, 'total_flos': 5839572172800.0, 'train_loss': 2.3774278950691223, 'epoch': 50.0})

In [38]:

# 추론
input_text = "### 질문: 우리집 강아지 이름은?\n### 답변:"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


### 질문: 우리집 강아지 이름은?
### 답변: 질문: 질문: 질문: 질문: 질문: 질문: 질문: �


In [39]:
#좀 더 해보자


# 사전 학습 모델 로드
model_id = "facebook/opt-350m"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(model_id)
base_model.config.pad_token_id = tokenizer.pad_token_id

# LoRA 설정 적용
lora_config = LoraConfig(
    r=16,  # 🔹 LoRA의 "랭크(rank)" 값입니다.
          # 학습할 파라미터 수를 줄이는 정도를 설정합니다.
          # r이 작을수록 계산이 가벼워지지만, 너무 작으면 성능이 떨어질 수 있습니다.

    lora_alpha=16,  # 🔹 LoRA가 학습한 정보를 얼마나 강하게 모델에 반영할지 정하는 값입니다.
                    # 일종의 "확대 비율"처럼 작용하며, 일반적으로 r과 함께 조정합니다.

    lora_dropout=0.01,  # 🔹 학습 중 일부 정보를 무작위로 버려 과적합을 막는 기술입니다.
                        # 0.05는 5% 확률로 드롭아웃이 일어나도록 설정한 것입니다.

    bias="none",  # 🔹 기존 모델의 편향(bias) 파라미터는 건드리지 않겠다는 뜻입니다.
                  # 즉, 오직 LoRA 레이어만 학습합니다.

    task_type="CAUSAL_LM"  # 🔹 이 설정이 적용될 작업의 유형입니다.
                           # "CAUSAL_LM"은 일반적인 언어 생성 모델(예: GPT)에서 사용됩니다.
)


model = get_peft_model(base_model, lora_config)

# 학습할 데이터 구성
data = {
    "text": [
        "### 질문: 우리집 강아지 이름은?\n### 답변: 순둥이",
    ]
}
dataset = Dataset.from_dict(data)

# 토큰화
def tokenize(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=64)

tokenized_dataset = dataset.map(tokenize)

# 데이터 콜레이터
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 학습 하이퍼파라미터
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    num_train_epochs=150,
    logging_steps=1,
    save_strategy="no",
    fp16=False,
    report_to="none"
)

# Trainer 구성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

<ipython-input-39-d086d351fb5e>:63: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
1,2.945800
2,2.876000
3,2.866500
4,2.723500
5,2.851600
6,2.770600
7,2.749400
8,2.666600
9,2.621900
10,2.622100


TrainOutput(global_step=150, training_loss=1.3883870216210683, metrics={'train_runtime': 16.5016, 'train_samples_per_second': 9.09, 'train_steps_per_second': 9.09, 'total_flos': 17564015001600.0, 'train_loss': 1.3883870216210683, 'epoch': 150.0})

In [40]:

# 추론
input_text = "### 질문: 우리집 강아지 이름은?\n### 답변:"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


### 질문: 우리집 강아지 이름은?
### 답변: 순둥이 순둥이 순둥이 순둥이 순둥이 순둥이 �


### 하이퍼 파라미터를 잘 튜닝하면 성능을 끌어올릴 수 있다
### 지금은 epoch 50 -> 150, fp 8 -> 16 으로 수정함
### 데이터가 너무 크면 결과가 안나올수도.. 개인이 파인튜닝은 어렵구나 결국 RAG나 랭채인으로...